In [3]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os 

#table_dropbox_path = "C:\Users\ande7\Dropbox\Apps\Overleaf\Thesis - Nash Bargaining\Tables"

def save_latex_table(latex_content, filename, caption, label):
    """Save a LaTeX table to the Dropbox path with proper formatting"""
    full_content = f"""
\\begin{{table}}[htbp]
\\centering
\\caption{{{caption}}}
\\label{{{label}}}
{latex_content}
\\end{{table}}
"""
    output_path = os.path.join(table_dropbox_path, filename)
    with open(output_path, 'w') as f:
        f.write(full_content)
    print(f"Table saved to: {output_path}")



In [4]:
# ===== EXAMPLE 1: Basic table from list data =====
print("EXAMPLE 1: Basic table from list data")

# Sample data as a list of lists
data = [
    ["Baseload", 45.2, 35.6, 0.85, "Low"],
    ["Pay-as-produced", 42.8, 37.2, 0.92, "Medium"],
    ["Collar", 40.1, 39.8, 0.96, "High"],
    ["2-sided CfD", 43.5, 36.4, 0.89, "Medium"]
]

headers = ["Contract Type", "Strike Price (€/MWh)", "Volume (MWh)", "Utility", "Risk"]

# Generate LaTeX table
latex_table = tabulate(data, headers, tablefmt="latex_booktabs")
print(latex_table)
print("\nTo use in LaTeX document:")
print(r"""
\begin{table}[htbp]
\centering
\caption{Comparison of Contract Types}
\label{tab:contract_comparison}
""" + latex_table + r"""
\end{table}
""")

# Usage example:
save_latex_table(
    latex_table, 
    "contract_comparison.tex", 
    "Comparison of Contract Types", 
    "tab:contract_comparison"
)

EXAMPLE 1: Basic table from list data
\begin{tabular}{lrrrl}
\toprule
 Contract Type   &   Strike Price (€/MWh) &   Volume (MWh) &   Utility & Risk   \\
\midrule
 Baseload        &                   45.2 &           35.6 &      0.85 & Low    \\
 Pay-as-produced &                   42.8 &           37.2 &      0.92 & Medium \\
 Collar          &                   40.1 &           39.8 &      0.96 & High   \\
 2-sided CfD     &                   43.5 &           36.4 &      0.89 & Medium \\
\bottomrule
\end{tabular}

To use in LaTeX document:

\begin{table}[htbp]
\centering
\caption{Comparison of Contract Types}
\label{tab:contract_comparison}
\begin{tabular}{lrrrl}
\toprule
 Contract Type   &   Strike Price (€/MWh) &   Volume (MWh) &   Utility & Risk   \\
\midrule
 Baseload        &                   45.2 &           35.6 &      0.85 & Low    \\
 Pay-as-produced &                   42.8 &           37.2 &      0.92 & Medium \\
 Collar          &                   40.1 &           39.8 &

NameError: name 'table_dropbox_path' is not defined

In [5]:

# --- Load your CSV data ---
# Replace with your actual CSV file path
earnings_df = pd.read_csv("earnings_risk_sensitivity.csv")
no_contract_g = pd.read_csv("no_contract_g.csv")['net_earnings_no_contract_true_G']
no_contract_l = pd.read_csv("no_contract_l.csv")['net_earnings_no_contract_true_L']

fixed_A_G = 0.5  # Example value
A_L_values = [0.1, 0.5, 0.9]  # Example values

# --- Filter for selected A_G and A_L values ---
df = earnings_df[(earnings_df['A_G'] == fixed_A_G) & (earnings_df['A_L'].isin(A_L_values))]

# --- Group by A_L and aggregate mean and std ---
summary = df.groupby('A_L').agg(
    **{'Mean G': ('Revenue_G', 'mean'),
       'Std G': ('Revenue_G', 'std'),
       'Mean L': ('Revenue_L', 'mean'),
       'Std L': ('Revenue_L', 'std')}
).reset_index()

# --- Format A_L for display ---
summary['A_L'] = summary['A_L'].apply(lambda x: f"A_L = {x}")
summary = summary.set_index('A_L')

# --- Add no-contract row ---
no_contract = pd.DataFrame({
    'Mean G': [no_contract_g.mean()],
    'Std G': [no_contract_g.std()],
    'Mean L': [no_contract_l.mean()],
    'Std L': [no_contract_l.std()]
}, index=['No Contract'])
summary = pd.concat([summary, no_contract])

# --- % Change row (from first to last A_L, not including No Contract) ---
if len(A_L_values) > 1:
    first, last = summary.iloc[0], summary.iloc[len(A_L_values)-1]
    pct_change = ((last - first) / first.abs() * 100).to_frame().T
    pct_change.index = ['% Change']
    summary = pd.concat([summary, pct_change])

# --- Format for display ---
for idx in summary.index[:-1]:  # skip % Change row
    summary.loc[idx] = summary.loc[idx].apply(lambda x: f"{x:.2f}")
if '% Change' in summary.index:
    summary.loc['% Change'] = summary.loc['% Change'].apply(lambda x: f"{float(x):.1f}%" if pd.notnull(x) else "")

summary.index.name = 'Risk Aversion'
print("\nGenerated Earnings Summary Table:")
print(summary)

# Optional: print as LaTeX table
latex_table = tabulate(summary, headers='keys', tablefmt="latex_booktabs")
print(latex_table)

FileNotFoundError: [Errno 2] No such file or directory: 'earnings_risk_sensitivity.csv'